<a href="https://colab.research.google.com/github/MuaddibMystic/beginning-bioinformatics/blob/main/Module04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Function to count how many times Pattern occurs in Text (including overlaps)

def CountOccurrences(Text, Pattern):
    """
    Count how many times the substring Pattern appears inside Text.
    Overlapping occurrences are included.

    Parameters:
        Text (str): The long DNA sequence.
        Pattern (str): The shorter DNA sequence to search for.

    Returns:
        int: Number of occurrences of Pattern in Text.
    """

    # Lengths of the two strings
    n = len(Text)
    m = len(Pattern)

    # Edge case: if the pattern is longer than the text, it cannot occur
    if m > n:
        return 0

    # Initialize counter
    count = 0

    # Loop over every possible starting position in Text where Pattern could fit
    # The last valid starting index is n - m
    for i in range(n - m + 1):
        # Extract substring of length m starting at position i
        substring = Text[i : i + m]

        # Check if the substring matches Pattern
        if substring == Pattern:
            count += 1  # Found a match, so increment count

    # Return the total number of matches found
    return count


# --------------------------
# Example usage (you can edit these inputs in Colab to test)
Text = "AAAAA"
Pattern = "AAA"

# Call the function
result = CountOccurrences(Text, Pattern)

# Print the result
print("Number of occurrences:", result)  # Expected output: 3


Number of occurrences: 3


In [ ]:
# --- Colab: upload + solve Rosalind "Count(Text, Pattern)" ---

# 1) Use Colab's file upload dialog to select your Rosalind dataset file
#    (e.g., "rosalind_subs.txt"). After running this cell, click "Choose Files".
from google.colab import files

uploaded = files.upload()  # prompts user to pick a file
if not uploaded:
    raise RuntimeError("No file uploaded. Please run the cell again and select a file.")

# Grab the first (and usually only) uploaded file name
filename = next(iter(uploaded))
print(f"Uploaded file: {filename}")

# 2) Read the file contents as text
raw_bytes = uploaded[filename]
text_data = raw_bytes.decode('utf-8', errors='replace')

# 3) Parse inputs from the file
# Rosalind typically provides:
#   Line 1: Text (DNA string)
#   Line 2: Pattern (DNA string)
# But to be robust, we’ll:
#   - Split on any whitespace,
#   - Use the first two non-empty tokens as Text and Pattern.
tokens = [tok.strip() for tok in text_data.split() if tok.strip()]

if len(tokens) < 2:
    raise ValueError(
        "Could not parse Text and Pattern from file.\n"
        "Make sure the file contains two DNA strings (Text on the first line, Pattern on the second line)."
    )

Text = tokens[0]
Pattern = tokens[1]

print(f"Parsed Text (length {len(Text)}): {Text[:60]}{'...' if len(Text) > 60 else ''}")
print(f"Parsed Pattern (length {len(Pattern)}): {Pattern}\n")

# 4) Define the counting function (overlapping occurrences allowed)
def CountOccurrences(Text, Pattern, verbose=False):
    """
    Count how many times Pattern appears in Text, allowing overlaps.
    If verbose=True, prints intermediate windows checked.
    """
    n = len(Text)
    m = len(Pattern)

    if m > n:
        if verbose:
            print("Pattern is longer than Text -> count = 0")
        return 0

    count = 0
    if verbose:
        print(f"Text length = {n}, Pattern length = {m}")
        print()

    for i in range(n - m + 1):
        substring = Text[i : i + m]
        if verbose:
            print(f"Checking substring [{i}:{i+m}] -> {substring}", end="")
        if substring == Pattern:
            count += 1
            if verbose:
                print("  -> MATCH (count += 1)")
        else:
            if verbose:
                print("  -> no match")
    if verbose:
        print(f"\nFinal count = {count}")
    return count

# 5) Run the solver
# Set verbose=True to see every checked window; False for just the final result.
verbose = False
result = CountOccurrences(Text, Pattern, verbose=verbose)

print(f"\nAnswer (Count(Text, Pattern)) = {result}")


Saving rosalind_ba1a.txt to rosalind_ba1a.txt
Uploaded file: rosalind_ba1a.txt
Parsed Text (length 1008): CCGGTGGCTGGTGGCTGGTGGCTGAATGTTTGGTGGCTGGTGGCTAGGTGGCTGGTGGCT...
Parsed Pattern (length 9): GGTGGCTGG


Answer (Count(Text, Pattern)) = 40


In [ ]:
# --- Colab: upload + solve "Most Frequent k-mers in a String" (Rosalind BA1B) ---

# Problem (for reference):
# Find the most frequent k-mers in a string.
# Given: A DNA string Text and an integer k.
# Return: All most frequent k-mers in Text (in any order).

from collections import defaultdict
from google.colab import files

def parse_text_and_k(raw_text):
    """
    Robustly parse Rosalind-style input:
      Line 1: Text (DNA string)
      Line 2: k (integer)
    If lines are messy, fall back to token-based parsing.
    """
    # Try simple line-based first
    lines = [ln.strip() for ln in raw_text.splitlines() if ln.strip()]
    if len(lines) >= 2:
        text_candidate = lines[0]
        try:
            k_candidate = int(lines[1])
            return text_candidate, k_candidate
        except ValueError:
            pass  # fall back to token approach

    # Fallback: token-based (handles extra whitespace/newlines)
    tokens = [tok.strip() for tok in raw_text.split() if tok.strip()]
    if len(tokens) < 2:
        raise ValueError(
            "Could not parse Text and k from the file. "
            "Expected first line = DNA string, second line = integer k."
        )
    text = tokens[0]
    try:
        k = int(tokens[1])
    except ValueError:
        raise ValueError("Second value is not an integer k.")
    return text, k


def most_frequent_kmers(Text, k, verbose=False):
    """
    Return all most frequent k-mers in Text (ties allowed).
    - Overlapping k-mers are counted.
    - If k <= 0 or k > len(Text), returns an empty list.
    """
    n = len(Text)
    if k <= 0 or k > n:
        if verbose:
            print(f"Invalid k: {k}. Must satisfy 1 <= k <= len(Text)={n}.")
        return []

    freq = defaultdict(int)

    # Slide window of length k over Text
    for i in range(n - k + 1):
        mer = Text[i : i + k]
        freq[mer] += 1
        if verbose:
            print(f"Window [{i}:{i+k}] -> {mer}, count = {freq[mer]}")

    # Determine the maximum frequency
    max_count = max(freq.values()) if freq else 0

    # Collect all k-mers that achieve the maximum frequency
    top_mers = [mer for mer, c in freq.items() if c == max_count]

    if verbose:
        print(f"\nMax frequency = {max_count}")
        print("Most frequent k-mers:")
        for mer in sorted(top_mers):
            print(f"  {mer} -> {max_count}")

    return top_mers


# === 1) Upload the Rosalind input file (e.g., "rosalind_ba1b.txt") ===
uploaded = files.upload()  # Opens the file chooser in Colab
if not uploaded:
    raise RuntimeError("No file uploaded. Please run the cell again and select a file.")

# Use the first uploaded file
filename = next(iter(uploaded))
print(f"Uploaded file: {filename}")

# === 2) Read file contents ===
raw_bytes = uploaded[filename]
raw_text = raw_bytes.decode('utf-8', errors='replace')

# === 3) Parse Text and k ===
Text, k = parse_text_and_k(raw_text)
print(f"Parsed Text (length {len(Text)}): {Text[:60]}{'...' if len(Text) > 60 else ''}")
print(f"Parsed k: {k}")

# === 4) Solve ===
verbose = False  # Set True to print every window & a small frequency report
answer_kmers = most_frequent_kmers(Text, k, verbose=verbose)

# === 5) Print the answer in Rosalind-friendly format (space-separated) ===
print("\nAnswer (most frequent k-mers):")
print(" ".join(answer_kmers))


Saving rosalind_ba1b.txt to rosalind_ba1b.txt
Uploaded file: rosalind_ba1b.txt
Parsed Text (length 813): GTGGCACCCGGCCCGGATTGAAACTGTGGCACCCAGGCCAAAATAGCTTTGAAACTCGGC...
Parsed k: 14

Answer (most frequent k-mers):
CGGCCCGGATTGAA GGCCCGGATTGAAA GCCCGGATTGAAAC CCCGGATTGAAACT


In [ ]:
# --- Colab: upload + solve "Clump Finding" (Rosalind BA1E) ---
# Problem (for reference):
# Find patterns forming clumps in a string.
# Given: A string Genome, and integers k, L, and t.
# Return: All distinct k-mers forming (L, t)-clumps in Genome.

from collections import defaultdict
from google.colab import files

def parse_genome_kLt(raw_text):
    """
    Parse Rosalind-style input:
      Line 1: Genome (DNA string)
      Line 2: k L t  (three integers, space-separated)
    Falls back to token-based parsing if lines are irregular.
    """
    # Try simple line-based parse first
    lines = [ln.strip() for ln in raw_text.splitlines() if ln.strip()]
    if len(lines) >= 2:
        genome_candidate = lines[0]
        parts = lines[1].split()
        if len(parts) >= 3:
            try:
                k = int(parts[0]); L = int(parts[1]); t = int(parts[2])
                return genome_candidate, k, L, t
            except ValueError:
                pass  # fall back

    # Fallback: token-based (handles extra whitespace/newlines)
    tokens = [tok.strip() for tok in raw_text.split() if tok.strip()]
    if len(tokens) < 4:
        raise ValueError(
            "Could not parse Genome, k, L, t from file. "
            "Expected first line = Genome string, second line = 'k L t'."
        )
    Genome = tokens[0]
    try:
        k = int(tokens[1]); L = int(tokens[2]); t = int(tokens[3])
    except ValueError:
        raise ValueError("k, L, t must be integers.")
    return Genome, k, L, t


def clump_finding(Genome, k, L, t, verbose=False):
    """
    Return all distinct k-mers forming (L, t)-clumps in Genome.
    A k-mer forms an (L, t)-clump if it appears at least t times
    within some window of length L.

    Sliding-window method:
      - Build frequency map for the first window [0:L].
      - Record any k-mer with count >= t.
      - Slide the window by 1 each time:
          * Decrement count of the leaving k-mer (start of previous window)
          * Increment count of the entering k-mer (end of new window)
      - Track any k-mer reaching count >= t in any window.

    Complexity: O(n) updates (two O(1) updates per slide), where n = len(Genome).
    """
    n = len(Genome)
    if k <= 0 or L <= 0 or t <= 0:
        if verbose:
            print(f"Invalid parameters: k={k}, L={L}, t={t}")
        return []
    if k > L or L > n:
        if verbose:
            print(f"No clumps possible: k={k}, L={L}, len(Genome)={n}")
        return []

    freq = defaultdict(int)
    clumps = set()

    # Helper to extract the k-mer at position i (assumes i+k <= n)
    def mer_at(i):
        return Genome[i:i+k]

    # Initialize counts for the first window [0:L]
    # Add all k-mers that start in [0, L-k]
    window_start = 0
    window_end = L
    for i in range(window_start, window_end - k + 1):
        m = mer_at(i)
        freq[m] += 1

    # Record any k-mer hitting threshold in the first window
    for m, c in freq.items():
        if c >= t:
            clumps.add(m)

    if verbose:
        print(f"Initialized first window [0:{L}]")
        print(f"Unique k-mers in first window: {len(freq)}")
        print(f"Clumps so far: {len(clumps)}")

    # Slide the window by 1 across Genome
    # Number of windows = n - L + 1; we already processed the first one (i=0)
    for w_start in range(1, n - L + 1):
        # Leaving k-mer starts at position (w_start-1)
        leave_pos = w_start - 1
        leaving = mer_at(leave_pos)
        freq[leaving] -= 1
        if freq[leaving] == 0:
            del freq[leaving]

        # Entering k-mer starts at position (w_start + (L - k))
        enter_pos = w_start + (L - k)
        entering = mer_at(enter_pos)
        freq[entering] += 1

        # If entering k-mer reaches threshold, record it
        if freq[entering] >= t:
            clumps.add(entering)

        if verbose and (w_start % 10000 == 0):
            print(f"Processed window starting at {w_start} "
                  f"(freq size={len(freq)}, clumps={len(clumps)})")

    return sorted(clumps)


# === 1) Upload the Rosalind input file (e.g., "rosalind_ba1e.txt") ===
uploaded = files.upload()  # Opens the file chooser in Colab
if not uploaded:
    raise RuntimeError("No file uploaded. Please run the cell again and select a file.")

# Use the first uploaded file
filename = next(iter(uploaded))
print(f"Uploaded file: {filename}")

# === 2) Read file contents ===
raw_bytes = uploaded[filename]
raw_text = raw_bytes.decode('utf-8', errors='replace')

# === 3) Parse Genome, k, L, t ===
Genome, k, L, t = parse_genome_kLt(raw_text)
print(f"Parsed Genome length: {len(Genome)}")
print(f"Parsed k={k}, L={L}, t={t}")

# === 4) Solve ===
verbose = False  # Set True for progress logs on huge inputs
answer_kmers = clump_finding(Genome, k, L, t, verbose=verbose)

# === 5) Print answer in Rosalind-friendly format (space-separated) ===
print("\nAnswer (k-mers forming (L, t)-clumps):")
print(" ".join(answer_kmers))


Saving rosalind_ba1e.txt to rosalind_ba1e.txt
Uploaded file: rosalind_ba1e.txt
Parsed Genome length: 8963
Parsed k=11, L=512, t=20

Answer (k-mers forming (L, t)-clumps):
ATCCGTTAGAA CTCGTAATTGT CTTGAACTCAG GATGTGGGGAG


In [ ]:
# --- Colab: upload + solve "Minimum Skew" (Rosalind BA1F) ---
# Problem (for reference):
# Find a position in a genome minimizing the skew.
# Given: A DNA string Genome.
# Return: All integer(s) i minimizing Skew(Prefix_i(Text)) for i in [0..|Genome|].
#
# Notes:
#   Skew(Prefix_i) = (# of 'G' in first i chars) - (# of 'C' in first i chars)
#   We consider i = 0 (empty prefix) up to i = |Genome| (full string).
#   A/T do not change skew. G increments by 1, C decrements by 1.

from google.colab import files

def parse_genome(raw_text):
    """
    Parse Rosalind-style input for Minimum Skew:
      - Typically a single line with the Genome string.
      - Be robust to stray whitespace/newlines.
    Returns:
      Genome (str)
    """
    # Prefer first non-empty line
    for line in raw_text.splitlines():
        s = line.strip()
        if s:
            return s
    # Fallback: any non-empty token
    tokens = [tok.strip() for tok in raw_text.split() if tok.strip()]
    if not tokens:
        raise ValueError("Could not find a Genome string in the uploaded file.")
    return tokens[0]


def minimum_skew_positions(Genome, verbose=False):
    """
    Compute all positions i (0..n) where skew is minimized.
    Skew update rules per char:
      - 'G' -> +1
      - 'C' -> -1
      - others (A/T/anything else) -> 0
    Returns:
      list[int]: positions achieving the minimum skew value.
    """
    n = len(Genome)
    g = Genome.upper()

    skew = 0                # skew at i = 0
    min_skew = 0            # current minimum skew seen so far
    positions = [0]         # i = 0 is included
    # Optional: store a few initial skews for debug printing if verbose
    if verbose:
        print(f"Genome length: {n}")
        print("Computing prefix skews...")

    # Walk i = 1..n; update skew using char at g[i-1]
    for i in range(1, n + 1):
        ch = g[i - 1]
        if ch == 'G':
            skew += 1
        elif ch == 'C':
            skew -= 1
        # A/T or other symbols: skew unchanged

        if skew < min_skew:
            min_skew = skew
            positions = [i]
        elif skew == min_skew:
            positions.append(i)

        if verbose and (i <= 30 or i == n):
            # Show a small prefix of progress, then the last step
            print(f"i={i:>4}, char={ch}, skew={skew}, min_skew={min_skew}")

    if verbose:
        print(f"\nMinimum skew value = {min_skew}")
        print(f"Positions achieving minimum skew (count={len(positions)}):")
        print(positions)

    return positions


# === 1) Upload the Rosalind input file (e.g., "rosalind_ba1f.txt") ===
uploaded = files.upload()  # Opens the file chooser in Colab
if not uploaded:
    raise RuntimeError("No file uploaded. Please run the cell again and select a file.")

# Use the first uploaded file
filename = next(iter(uploaded))
print(f"Uploaded file: {filename}")

# === 2) Read file contents ===
raw_bytes = uploaded[filename]
raw_text = raw_bytes.decode('utf-8', errors='replace')

# === 3) Parse Genome ===
Genome = parse_genome(raw_text)
print(f"Parsed Genome length: {len(Genome)}")

# === 4) Solve ===
verbose = False  # Set to True for a small running log / sanity checks
positions = minimum_skew_positions(Genome, verbose=verbose)

# === 5) Print answer in Rosalind-friendly format (space-separated) ===
print("\nAnswer (positions of minimum skew):")
print(" ".join(str(i) for i in positions))


Saving rosalind_ba1f.txt to rosalind_ba1f.txt
Uploaded file: rosalind_ba1f.txt
Parsed Genome length: 81587

Answer (positions of minimum skew):
72618 72621 72622


In [ ]:
# --- Colab: upload + solve "Hamming Distance" (Rosalind HAMM) ---
# Problem (for reference):
# Compute the Hamming distance between two DNA strings.
#
# Given: Two DNA strings of equal length.
# Return: An integer value representing the Hamming distance.

from google.colab import files

def parse_two_dna_strings(raw_text):
    """
    Parse Rosalind-style input:
      Line 1: DNA string 1
      Line 2: DNA string 2
    Returns:
      (s1, s2) as strings
    """
    # Get non-empty lines
    lines = [ln.strip() for ln in raw_text.splitlines() if ln.strip()]
    if len(lines) >= 2:
        return lines[0], lines[1]
    # Fallback: use tokens
    tokens = [tok.strip() for tok in raw_text.split() if tok.strip()]
    if len(tokens) < 2:
        raise ValueError("Expected at least two DNA strings in the file.")
    return tokens[0], tokens[1]


def hamming_distance(s1, s2, verbose=False):
    """
    Compute the Hamming distance between two equal-length strings.
    Returns:
      int = number of mismatched positions
    """
    if len(s1) != len(s2):
        raise ValueError(f"Strings must be equal length, got {len(s1)} and {len(s2)}")
    distance = 0
    for i, (a, b) in enumerate(zip(s1, s2)):
        if a != b:
            distance += 1
            if verbose and i < 30:
                print(f"Mismatch at index {i}: {a} vs {b}")
    if verbose:
        print(f"\nTotal Hamming distance = {distance}")
    return distance


# === 1) Upload the Rosalind input file (e.g., "rosalind_hamm.txt") ===
uploaded = files.upload()  # Opens the file chooser in Colab
if not uploaded:
    raise RuntimeError("No file uploaded. Please run the cell again and select a file.")

# Use the first uploaded file
filename = next(iter(uploaded))
print(f"Uploaded file: {filename}")

# === 2) Read file contents ===
raw_bytes = uploaded[filename]
raw_text = raw_bytes.decode('utf-8', errors='replace')

# === 3) Parse the two DNA strings ===
s1, s2 = parse_two_dna_strings(raw_text)
print(f"Parsed DNA strings of length {len(s1)} and {len(s2)}")

# === 4) Solve ===
verbose = False  # Set to True to see mismatches logged
result = hamming_distance(s1, s2, verbose=verbose)

# === 5) Print answer in Rosalind-friendly format ===
print("\nAnswer (Hamming distance):")
print(result)


Saving rosalind_ba1g.txt to rosalind_ba1g.txt
Uploaded file: rosalind_ba1g.txt
Parsed DNA strings of length 1060 and 1060

Answer (Hamming distance):
782


In [ ]:
# --- Colab: upload + solve "Neighbors(Pattern, d)" (Rosalind BA1N) ---
# Problem (for reference):
# Find all the neighbors of a pattern.
# Given: A DNA string Pattern and an integer d.
# Return: The collection of strings Neighbors(Pattern, d).
#
# Notes:
#   - Neighbors(Pattern, d) = all strings over {A,C,G,T} of the same length as Pattern
#     whose Hamming distance from Pattern is ≤ d.
#   - Output order doesn't matter. Rosalind typically accepts any order.

from google.colab import files

ALPHABET = ('A', 'C', 'G', 'T')

def parse_pattern_and_d(raw_text):
    """
    Parse Rosalind-style input:
      Line 1: Pattern (DNA string)
      Line 2: d (integer)
    Falls back to token parsing if needed.
    """
    lines = [ln.strip() for ln in raw_text.splitlines() if ln.strip()]
    if len(lines) >= 2:
        pat = lines[0]
        try:
            d = int(lines[1])
            return pat, d
        except ValueError:
            pass

    tokens = [tok.strip() for tok in raw_text.split() if tok.strip()]
    if len(tokens) < 2:
        raise ValueError(
            "Could not parse Pattern and d. Expected:\n"
            "Line 1 = Pattern (DNA), Line 2 = integer d."
        )
    pattern = tokens[0]
    try:
        d = int(tokens[1])
    except ValueError:
        raise ValueError("Second value is not an integer d.")
    return pattern, d


def hamming_distance(s1, s2):
    """Return the Hamming distance between two equal-length strings."""
    if len(s1) != len(s2):
        raise ValueError("Strings must be equal length.")
    return sum(a != b for a, b in zip(s1, s2))


def neighbors(pattern, d):
    """
    Recursive construction of Neighbors(Pattern, d).
    Standard textbook/BA1N approach:
      - If d == 0: only the pattern itself.
      - If len(pattern) == 1: all nucleotides (A,C,G,T).
      - Otherwise:
          * Compute neighbors of the suffix.
          * If suffix-neighbor differs from suffix by < d, we can vary the first char freely.
          * Else we must keep the first char unchanged.
    Returns a Python set of strings.
    """
    if d == 0:
        return {pattern}
    if len(pattern) == 1:
        return set(ALPHABET)

    neighborhood = set()
    suffix = pattern[1:]
    suffix_neighbors = neighbors(suffix, d)

    for sn in suffix_neighbors:
        # how many mismatches used in the suffix
        dist = hamming_distance(suffix, sn)
        if dist < d:
            # we still have "budget" to change the first char arbitrarily
            for x in ALPHABET:
                neighborhood.add(x + sn)
        else:
            # no budget left: keep first char as is
            neighborhood.add(pattern[0] + sn)

    return neighborhood


# === 1) Upload the Rosalind input file (e.g., "rosalind_ba1n.txt") ===
uploaded = files.upload()  # Opens file chooser in Colab
if not uploaded:
    raise RuntimeError("No file uploaded. Please run the cell again and select a file.")

# Use the first uploaded file
filename = next(iter(uploaded))
print(f"Uploaded file: {filename}")

# === 2) Read file contents ===
raw_bytes = uploaded[filename]
raw_text = raw_bytes.decode('utf-8', errors='replace')

# === 3) Parse Pattern and d ===
Pattern, d = parse_pattern_and_d(raw_text)
print(f"Parsed Pattern (len={len(Pattern)}): {Pattern}")
print(f"Parsed d: {d}")

# Basic sanity checks
if d < 0:
    raise ValueError("d must be a non-negative integer.")

# === 4) Solve ===
answer_set = neighbors(Pattern, d)

# === 5) Print the answer in Rosalind-friendly format (space-separated) ===
print("\nAnswer (Neighbors(Pattern, d)):")
print(" ".join(answer_set))


Saving rosalind_ba1n.txt to rosalind_ba1n.txt
Uploaded file: rosalind_ba1n.txt
Parsed Pattern (len=11): TTACGAGACTT
Parsed d: 2

Answer (Neighbors(Pattern, d)):
TTACTGGACTT TTAGGTGACTT TGACGAGACCT TTACGCGACTC ATACGAGAATT TTGTGAGACTT TTACGACACTG CTACGAGTCTT TTATGAGACTG TTGCGAGAATT CTACGAGACCT ATACGAGACTA TTCCGATACTT CTACAAGACTT TGACGAGACTT TTACGAAACCT TTAATAGACTT TTACGAGACAT TCACGAGCCTT TGAAGAGACTT TGACGAAACTT TTCCGGGACTT TAACGAGAGTT TTAACAGACTT TTACGACACTC TTACGAGAGTC TTACGCGACAT TTACGAGGCTG TTACCAGACAT TCACGAGAGTT TGACGAGTCTT TTACGATACAT TTACGAGGTTT TTATGAGCCTT TTACTAGAGTT TTCCGCGACTT TTACGAGTTTT TTACGTGACTC TGACGGGACTT TTTCGAGACTC TTACGAGATTA TTACGATCCTT TTTCGAGCCTT CTACGAGACGT ATACGAGACTT TTACGTGACTT TAACGACACTT TACCGAGACTT TTGCGAGACAT TTGCGGGACTT TTGCGAGATTT TTACGGGACCT TTACGTGATTT TTACGACCCTT TTAAGAGATTT GTACCAGACTT TTACGAGCATT TTTCGTGACTT TTAAGAGACAT ATACGTGACTT TTAGGAGAGTT GTATGAGACTT TTCCGAGAATT TTACGCGACGT TTTCGCGACTT TTCGGAGACTT CTACGTGACTT TTACGCAACTT TTTCGACACTT TTACTTGACTT

In [ ]:
# --- Colab: upload + solve "Neighbors(Pattern, d)" (Rosalind BA1N) ---
# Task:
# Generate the d-neighborhood of a string.
# Given: A DNA string Pattern and an integer d.
# Return: The collection of strings Neighbors(Pattern, d) (each on its own line).

from google.colab import files

ALPHABET = ('A', 'C', 'G', 'T')

def parse_pattern_and_d(raw_text):
    """
    Parse input as:
      Line 1: Pattern (DNA string)
      Line 2: d (integer)
    Robust to extra whitespace/newlines.
    """
    lines = [ln.strip() for ln in raw_text.splitlines() if ln.strip()]
    if len(lines) >= 2:
        pat = lines[0].strip()
        d = int(lines[1].strip())
        return pat, d

    # Fallback: token-based parsing
    tokens = [tok.strip() for tok in raw_text.split() if tok.strip()]
    if len(tokens) < 2:
        raise ValueError("Expected two lines: Pattern on line 1, integer d on line 2.")
    return tokens[0], int(tokens[1])


def hamming_distance(s1, s2):
    """Return the Hamming distance between two equal-length strings."""
    if len(s1) != len(s2):
        raise ValueError("Strings must be equal length for Hamming distance.")
    return sum(a != b for a, b in zip(s1, s2))


def neighbors(pattern, d):
    """
    Recursive construction of Neighbors(Pattern, d):
      - If d == 0: return {pattern}
      - If len(pattern) == 1: return {'A','C','G','T'}
      - Else:
          * Generate neighbors of the suffix (pattern[1:])
          * If suffix-neighbor differs from suffix by < d, vary the first char freely
          * Otherwise, keep the first char the same
    Returns a set of strings.
    """
    if d == 0:
        return {pattern}
    if len(pattern) == 1:
        return set(ALPHABET)

    neighborhood = set()
    suffix = pattern[1:]
    suffix_neighbors = neighbors(suffix, d)

    for sn in suffix_neighbors:
        # mismatches used in the suffix
        dist = hamming_distance(suffix, sn)
        if dist < d:
            # we can still mutate the first character
            for x in ALPHABET:
                neighborhood.add(x + sn)
        else:
            # mutation budget spent in suffix: keep first char
            neighborhood.add(pattern[0] + sn)

    return neighborhood


# === 1) Upload the Rosalind input file (e.g., "rosalind_ba1n.txt") ===
uploaded = files.upload()  # Opens file chooser in Colab
if not uploaded:
    raise RuntimeError("No file uploaded. Please run the cell again and select a file.")

# Use the first uploaded file
filename = next(iter(uploaded))
print(f"Uploaded file: {filename}")

# === 2) Read file contents ===
raw_bytes = uploaded[filename]
raw_text = raw_bytes.decode('utf-8', errors='replace')

# === 3) Parse Pattern and d ===
Pattern, d = parse_pattern_and_d(raw_text)
Pattern = Pattern.upper().strip()
print(f"Parsed Pattern (len={len(Pattern)}): {Pattern}")
print(f"Parsed d: {d}")

if d < 0:
    raise ValueError("d must be a non-negative integer.")

# === 4) Solve ===
answer_set = neighbors(Pattern, d)

# === 5) Print each neighbor on its own line (Rosalind-friendly) ===
# Order doesn't matter to Rosalind; sorting makes output deterministic.
print("\nAnswer (Neighbors(Pattern, d)):")
for mer in sorted(answer_set):
    print(mer)


Saving rosalind_ba1n (1).txt to rosalind_ba1n (1).txt
Uploaded file: rosalind_ba1n (1).txt
Parsed Pattern (len=9): ATCTGTCGT
Parsed d: 2

Answer (Neighbors(Pattern, d)):
AAATGTCGT
AACAGTCGT
AACCGTCGT
AACGGTCGT
AACTATCGT
AACTCTCGT
AACTGACGT
AACTGCCGT
AACTGGCGT
AACTGTAGT
AACTGTCAT
AACTGTCCT
AACTGTCGA
AACTGTCGC
AACTGTCGG
AACTGTCGT
AACTGTCTT
AACTGTGGT
AACTGTTGT
AACTTTCGT
AAGTGTCGT
AATTGTCGT
ACATGTCGT
ACCAGTCGT
ACCCGTCGT
ACCGGTCGT
ACCTATCGT
ACCTCTCGT
ACCTGACGT
ACCTGCCGT
ACCTGGCGT
ACCTGTAGT
ACCTGTCAT
ACCTGTCCT
ACCTGTCGA
ACCTGTCGC
ACCTGTCGG
ACCTGTCGT
ACCTGTCTT
ACCTGTGGT
ACCTGTTGT
ACCTTTCGT
ACGTGTCGT
ACTTGTCGT
AGATGTCGT
AGCAGTCGT
AGCCGTCGT
AGCGGTCGT
AGCTATCGT
AGCTCTCGT
AGCTGACGT
AGCTGCCGT
AGCTGGCGT
AGCTGTAGT
AGCTGTCAT
AGCTGTCCT
AGCTGTCGA
AGCTGTCGC
AGCTGTCGG
AGCTGTCGT
AGCTGTCTT
AGCTGTGGT
AGCTGTTGT
AGCTTTCGT
AGGTGTCGT
AGTTGTCGT
ATAAGTCGT
ATACGTCGT
ATAGGTCGT
ATATATCGT
ATATCTCGT
ATATGACGT
ATATGCCGT
ATATGGCGT
ATATGTAGT
ATATGTCAT
ATATGTCCT
ATATGTCGA
ATATGTCGC
ATATGTCGG
ATATGTCGT
ATATGTCTT
ATATGTGGT


In [ ]:
# --- Colab: upload + solve "Approximate Pattern Matching" (Rosalind BA1H) ---
# Problem (for reference):
# Find all approximate occurrences of a pattern in a string.
# Given: Strings Pattern and Text along with an integer d.
# Return: All starting positions where Pattern appears in Text with at most d mismatches.

from google.colab import files

def parse_pattern_text_d(raw_text):
    """
    Parse Rosalind-style input:
      Line 1: Pattern (DNA string)
      Line 2: Text (DNA string)
      Line 3: d (integer)
    Robust to extra whitespace/newlines.
    """
    lines = [ln.strip() for ln in raw_text.splitlines() if ln.strip()]
    if len(lines) >= 3:
        pat = lines[0]
        txt = lines[1]
        d   = int(lines[2])
        return pat, txt, d

    # Fallback: token-based parsing
    tokens = [tok.strip() for tok in raw_text.split() if tok.strip()]
    if len(tokens) < 3:
        raise ValueError("Expected Pattern, Text, and integer d (three values).")
    return tokens[0], tokens[1], int(tokens[2])


def hamming_distance_limited(a, b, limit=None):
    """
    Hamming distance with optional early stopping:
      - If 'limit' is provided and mismatches exceed it, stop early.
    """
    if len(a) != len(b):
        raise ValueError("Strings must be equal length for Hamming distance.")
    mismatches = 0
    for x, y in zip(a, b):
        if x != y:
            mismatches += 1
            if limit is not None and mismatches > limit:
                return mismatches  # early exit
    return mismatches


def approximate_match_positions(Pattern, Text, d, verbose=False):
    """
    Return all starting indices i where HammingDistance(Pattern, Text[i:i+m]) <= d.
    Overlapping occurrences allowed. Indices are 0-based (Rosalind expects 0-based).
    """
    m = len(Pattern)
    n = len(Text)
    if m > n:
        return []

    positions = []
    for i in range(n - m + 1):
        window = Text[i:i+m]
        # Early-stop Hamming distance using 'd' as the threshold
        dist = hamming_distance_limited(Pattern, window, limit=d)
        if dist <= d:
            positions.append(i)
        if verbose and i < 20:
            print(f"i={i:>4} window={window} dist={dist} -> {'KEEP' if dist <= d else 'skip'}")
    return positions


# === 1) Upload the Rosalind input file (e.g., "rosalind_ba1h.txt") ===
uploaded = files.upload()  # Opens the file chooser in Colab
if not uploaded:
    raise RuntimeError("No file uploaded. Please run the cell again and select a file.")

# Use the first uploaded file
filename = next(iter(uploaded))
print(f"Uploaded file: {filename}")

# === 2) Read file contents ===
raw_bytes = uploaded[filename]
raw_text = raw_bytes.decode('utf-8', errors='replace')

# === 3) Parse Pattern, Text, d ===
Pattern, Text, d = parse_pattern_text_d(raw_text)
print(f"Parsed Pattern (len={len(Pattern)}), Text (len={len(Text)}), d={d}")

# === 4) Solve ===
verbose = False  # Set True to log the first ~20 windows for sanity-check
positions = approximate_match_positions(Pattern, Text, d, verbose=verbose)

# === 5) Print answer in Rosalind-friendly format (space-separated positions) ===
print("\nAnswer (starting positions with ≤ d mismatches):")
print(" ".join(map(str, positions)))


Saving rosalind_ba1h.txt to rosalind_ba1h.txt
Uploaded file: rosalind_ba1h.txt
Parsed Pattern (len=9), Text (len=17680), d=6

Answer (starting positions with ≤ d mismatches):
1 2 5 7 9 11 12 14 15 17 19 20 21 23 29 30 31 32 34 36 38 47 49 51 53 57 59 64 66 68 70 74 78 80 84 85 87 89 98 105 108 110 113 114 115 116 124 127 129 131 135 136 137 140 142 145 147 149 151 153 157 159 163 165 167 168 170 171 172 173 175 177 180 182 185 187 189 191 193 197 198 204 210 211 216 217 221 227 228 233 235 237 238 240 241 246 248 252 253 257 263 267 276 277 282 284 288 293 295 297 303 304 306 307 308 313 318 323 324 325 327 329 330 331 335 337 339 341 344 346 347 348 349 355 362 369 371 373 374 380 381 382 388 390 391 392 396 398 400 405 407 409 411 412 417 418 419 423 425 426 429 433 435 439 444 446 449 451 453 455 460 461 462 463 465 467 472 474 476 478 479 481 482 483 484 485 486 488 491 492 494 496 497 500 502 506 508 510 512 518 520 527 529 531 533 535 538 540 541 543 546 548 552 555 558 560 562 5

In [ ]:
# --- Colab: upload + solve "Frequent Words with Mismatches" (Rosalind BA1I) ---
# Problem:
# Given: A string Text and integers k and d.
# Return: All most frequent k-mers with up to d mismatches in Text (ties allowed, any order).

from collections import defaultdict
from google.colab import files

ALPHABET = ('A', 'C', 'G', 'T')

def parse_text_k_d(raw_text):
    """
    Parse Rosalind-style input:
      Line 1: Text
      Line 2: k d   (two integers, space-separated)
    Robust to extra whitespace/newlines.
    """
    lines = [ln.strip() for ln in raw_text.splitlines() if ln.strip()]
    if len(lines) >= 2:
        text = lines[0]
        parts = lines[1].split()
        if len(parts) >= 2:
            k = int(parts[0]); d = int(parts[1])
            return text, k, d

    # Fallback: token-based parsing
    tokens = [tok.strip() for tok in raw_text.split() if tok.strip()]
    if len(tokens) < 3:
        raise ValueError("Expected: Text on line 1; 'k d' on line 2.")
    text = tokens[0]
    k = int(tokens[1]); d = int(tokens[2])
    return text, k, d


def neighbors(pattern, d, _memo=None):
    """
    Neighbors(Pattern, d): set of all strings at Hamming distance <= d.
    Recursive construction with memoization for speed.
    """
    if _memo is None:
        _memo = {}
    key = (pattern, d)
    if key in _memo:
        return _memo[key]

    if d == 0:
        res = {pattern}
    elif len(pattern) == 1:
        res = set(ALPHABET)
    else:
        neighborhood = set()
        suffix = pattern[1:]
        for sn in neighbors(suffix, d, _memo):
            # Count mismatches used in the suffix by comparing chars
            # Fast path without separate hamming fn
            suffix_mismatches = sum(1 for a, b in zip(suffix, sn) if a != b)
            if suffix_mismatches < d:
                for x in ALPHABET:
                    neighborhood.add(x + sn)
            else:
                neighborhood.add(pattern[0] + sn)
        res = neighborhood

    _memo[key] = res
    return res


def frequent_words_with_mismatches(Text, k, d, verbose=False):
    """
    Countd(Text, Pattern) via neighbor aggregation:
      - For each k-mer in Text, add +1 to all its neighbors within d mismatches.
      - The most frequent keys in this aggregated map are the answer.
    Complexity depends on the size of the d-neighborhood (~O(n * Σ_{i=0..d} C(k,i)*3^i)).
    """
    n = len(Text)
    if k <= 0 or d < 0 or k > n:
        return []

    counts = defaultdict(int)
    memo_neighbors = {}
    # Optional: deduplicate same k-mers in Text to avoid recomputing neighbors repeatedly
    # but we still need to add counts for each occurrence, so we cache the neighbor SET per k-mer.
    for i in range(n - k + 1):
        kmer = Text[i:i+k]
        if (kmer, d) not in memo_neighbors:
            memo_neighbors[(kmer, d)] = neighbors(kmer, d)
        for nb in memo_neighbors[(kmer, d)]:
            counts[nb] += 1

        if verbose and i % 10000 == 0 and i > 0:
            print(f"Processed {i} / {n - k + 1} windows... unique patterns so far: {len(counts)}")

    if not counts:
        return []

    max_count = max(counts.values())
    result = [p for p, c in counts.items() if c == max_count]

    if verbose:
        print(f"Max count = {max_count}, total candidates = {len(counts)}, answers = {len(result)}")

    return result


# === 1) Upload the Rosalind input file (e.g., "rosalind_ba1i.txt") ===
uploaded = files.upload()  # Opens the file chooser in Colab
if not uploaded:
    raise RuntimeError("No file uploaded. Please run the cell again and select a file.")

# Use the first uploaded file
filename = next(iter(uploaded))
print(f"Uploaded file: {filename}")

# === 2) Read file contents ===
raw_bytes = uploaded[filename]
raw_text = raw_bytes.decode('utf-8', errors='replace')

# === 3) Parse Text, k, d ===
Text, k, d = parse_text_k_d(raw_text)
print(f"Parsed: len(Text)={len(Text)}, k={k}, d={d}")

# === 4) Solve ===
verbose = False  # Set True for occasional progress logs on huge inputs
answers = frequent_words_with_mismatches(Text, k, d, verbose=verbose)

# === 5) Print the answer in Rosalind-friendly format (space-separated) ===
print("\nAnswer (most frequent k-mers with up to d mismatches):")
print(" ".join(answers))


Saving rosalind_ba1i.txt to rosalind_ba1i.txt
Uploaded file: rosalind_ba1i.txt
Parsed: len(Text)=909, k=5, d=2

Answer (most frequent k-mers with up to d mismatches):
CCCGC


In [ ]:
# --- Colab: upload + solve "Frequent Words with Mismatches and Reverse Complements" (BA1J) ---
# Problem:
# Find the most frequent k-mers (with mismatches and reverse complements) in a DNA string.
# Given: Text, k, d
# Return: All k-mers Pattern maximizing Count_d(Text, Pattern) + Count_d(Text, rc(Pattern))

from collections import defaultdict
from google.colab import files

ALPHABET = ('A', 'C', 'G', 'T')

def parse_text_k_d(raw_text):
    """
    Input format:
      Line 1: Text (DNA)
      Line 2: k d
    Robust to extra whitespace/newlines.
    """
    lines = [ln.strip() for ln in raw_text.splitlines() if ln.strip()]
    if len(lines) >= 2:
        text = lines[0]
        parts = lines[1].split()
        if len(parts) >= 2:
            k = int(parts[0]); d = int(parts[1])
            return text, k, d

    tokens = [tok.strip() for tok in raw_text.split() if tok.strip()]
    if len(tokens) < 3:
        raise ValueError("Expected: <Text> on line 1; 'k d' on line 2.")
    text = tokens[0]; k = int(tokens[1]); d = int(tokens[2])
    return text, k, d


def reverse_complement(s):
    comp = {'A':'T','T':'A','C':'G','G':'C'}
    return "".join(comp.get(c, c) for c in reversed(s))


def neighbors(pattern, d, _memo=None):
    """
    Neighbors(Pattern, d): all strings at Hamming distance <= d from pattern.
    Classic recursive construction with memoization.
    """
    if _memo is None:
        _memo = {}
    key = (pattern, d)
    if key in _memo:
        return _memo[key]

    if d == 0:
        res = {pattern}
    elif len(pattern) == 1:
        res = set(ALPHABET)
    else:
        neighborhood = set()
        suffix = pattern[1:]
        suffix_neighbors = neighbors(suffix, d, _memo)
        for sn in suffix_neighbors:
            # mismatches already used in the suffix
            used = sum(1 for a, b in zip(suffix, sn) if a != b)
            if used < d:
                for x in ALPHABET:
                    neighborhood.add(x + sn)
            else:
                neighborhood.add(pattern[0] + sn)
        res = neighborhood

    _memo[key] = res
    return res


def frequent_words_mismatch_with_rc(Text, k, d, verbose=False):
    """
    Count_d via neighbor aggregation, then fold reverse complements:
      counts[p] = approximate occurrences of p in Text
      score(p) = counts[p] + counts[rc(p)]
    Return all p with maximal score(p).
    """
    n = len(Text)
    if k <= 0 or d < 0 or k > n:
        return []

    counts = defaultdict(int)
    memo_neighbors = {}

    # Slide k-window over Text; for each window k-mer, add +1 to all its neighbors
    for i in range(n - k + 1):
        kmer = Text[i:i+k]
        key = (kmer, d)
        if key not in memo_neighbors:
            memo_neighbors[key] = neighbors(kmer, d)
        for nb in memo_neighbors[key]:
            counts[nb] += 1

        if verbose and i % 10000 == 0 and i > 0:
            print(f"Processed {i}/{n - k + 1} windows; unique candidates = {len(counts)}")

    # Fold reverse-complement counts
    best = 0
    result = set()
    for p in counts.keys():
        total = counts[p] + counts.get(reverse_complement(p), 0)
        if total > best:
            best = total
            result = {p}
        elif total == best:
            result.add(p)

    if verbose:
        print(f"Best combined count = {best}, answers = {len(result)}")

    return sorted(result)


# === 1) Upload the Rosalind input file (e.g., "rosalind_ba1j.txt") ===
uploaded = files.upload()
if not uploaded:
    raise RuntimeError("No file uploaded. Please run the cell again and select a file.")

# === 2) Read & parse ===
filename = next(iter(uploaded))
raw_text = uploaded[filename].decode('utf-8', errors='replace')
Text, k, d = parse_text_k_d(raw_text)
print(f"Parsed: len(Text)={len(Text)}, k={k}, d={d}")

# === 3) Solve ===
verbose = False  # Set True for progress logs on very large inputs
answers = frequent_words_mismatch_with_rc(Text, k, d, verbose=verbose)

# === 4) Print answer (space-separated, Rosalind-friendly) ===
print("\nAnswer (most frequent k-mers with mismatches + reverse complements):")
print(" ".join(answers))


Saving rosalind_ba1j.txt to rosalind_ba1j.txt
Parsed: len(Text)=995, k=7, d=2

Answer (most frequent k-mers with mismatches + reverse complements):
AGGGGAA TTCCCCT
